In [1]:
from platform import python_version
print(python_version())

3.7.4


https://www.criminaljustice.ny.gov/crimnet/ojsa/adult-arrest-demographics/2018/index.html

In [7]:
URL = 'https://www.criminaljustice.ny.gov/crimnet/ojsa/adult-arrest-demographics/2018/all.xls'
filename = 'all.xls'
from urllib.request import urlretrieve
urlretrieve(URL,filename)

('all.xls', <http.client.HTTPMessage at 0x103ca7710>)

In [8]:
# make sure the downloaded file is ignored so it's not in the github folder